# Hawaii - A Climate Analysis And Exploration
### For data between August 23, 2016 - August 23, 2017
---

In [2]:
# Import dependencies

%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

## Reflect Tables into SQLAlchemy ORM

In [3]:
# Set up query engine. 'echo=True is the default - will keep a log of activities'

engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [4]:
# Reflect an existing database into a new model
Base = automap_base()
# Reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [6]:
# Another way to get table names from SQL-lite
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']